In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
builder: SparkSession.Builder = SparkSession.builder
from pyspark.sql.functions import udf, col, lit, min, max, avg, desc, round, count, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType, TimestampType
import pandas as pd
import numpy as np
from datetime import datetime

# Chemins
DATA_DIR = "../data"
OUTPUT_DIR = "../output"
PARQUET_DIR = "../output/consommations_clean/"

# création des dossiers si non existant
output_dir = "../data/output"
output_dir_graphics = "../data/output/graphics"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_graphics, exist_ok=True)


### Chargement des données

In [2]:
# Creation de la session spark
# spark = builder.master('local').appName("demo_rdd").getOrCreate()

# df_raw_consommations = spark.read\
#             .option("header", "true")\
#             .option("inferSchema", "true")\
#             .option("sep", ",")\
#             .csv("../data/consommation_clean.csv")


# df_raw_consommations.show(5)

# chargement des donnees 
df_batiments_raw = pd.read_csv(f"{DATA_DIR}/batiments.csv")

print(f"Shape: {df_batiments_raw.shape}")
print(f"\nColonnes: {df_batiments_raw.columns.tolist()}")
df_batiments_raw.head(10)



Shape: (146, 8)

Colonnes: ['batiment_id', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen']


,batiment_id,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen
0,BAT0001,Ecole Paris 1,ecole,Paris,1926,1978,E,225
1,BAT0002,Ecole Paris 2,ecole,Paris,1156,2004,C,402
2,BAT0003,Ecole Paris 3,ecole,Paris,1695,2014,D,219
3,BAT0004,Mediatheque Paris 4,mediatheque,Paris,907,2019,C,121
4,BAT0005,Piscine Paris 5,piscine,Paris,3913,1950,G,242
5,BAT0006,Mairie Paris 6,mairie,Paris,1165,1985,E,63
6,BAT0007,Gymnase Paris 7,gymnase,Paris,1209,1998,C,73
7,BAT0008,Gymnase Paris 8,gymnase,Paris,2236,1955,G,117
8,BAT0009,Ecole Lyon 9,ecole,Lyon,2350,2020,B,140
9,BAT0010,Mediatheque Lyon 10,mediatheque,Lyon,1240,1974,G,103


In [ ]:
import os

if os.path.exists(PARQUET_DIR):
    df_pollution = pd.read_parquet(PARQUET_DIR)
    print(f"Donnees pollution chargees: {len(df_pollution):,} lignes")
    df_pollution.head()
    

### calcul intensité énergétique (kWh/m2)

### Identifier les batiments hors norme (>3x la mediane de leur categorie)

### Calculer les totaux par commune et par type de batiment